<a href="https://colab.research.google.com/github/gnanamathi26/Deep-learning/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dropout
import cv2
from keras.layers import Dropout,ZeroPadding2D
from sklearn.metrics import f1_score
from keras.preprocessing.image import ImageDataGenerator, load_img


In [ ]:
Normal=os.listdir('/content/drive/MyDrive/tuberdataset/Normal')
Tuber=os.listdir('/content/drive/MyDrive/tuberdataset/Tuberculosis')

In [ ]:
data= np.concatenate([Normal,Tuber])

In [ ]:
data

In [ ]:
target_x=np.full(len(Normal),1)

In [ ]:
target_x1=np.full(len(Tuber),0)

In [ ]:
data_target=np.concatenate([target_x,target_x1])

In [ ]:
X_data = []

In [ ]:
for file in Normal:
 img = cv2.imread('/content/drive/MyDrive/tuberdataset/Normal/' + file)
 mri = cv2.resize(img, (255,255)) #convert all img to 256px
 b, g , r = cv2.split(mri) #convert B G R format to R G B format 
 img = cv2.merge([r, g, b])
 X_data.append(img) #add to li

In [ ]:
for file in Tuber:
 img = cv2.imread('/content/drive/MyDrive/tuberdataset/Tuberculosis/' + file)
 mri = cv2.resize(img, (255,255)) #convert all img to 256px
 b, g , r = cv2.split(mri) #convert B G R format to R G B format 
 img = cv2.merge([r, g, b])
 X_data.append(img) #add to li

In [ ]:
X=np.squeeze(X_data)

In [ ]:
X=X.astype('float32')
X/=255


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,data_target,test_size=0.1,random_state=3)


In [ ]:
X_train2,X_val,y_train2,y_val=train_test_split(X_train,y_train,test_size=0.1,random_state=3)

In [ ]:
print(X_train2.shape)
print(X_val.shape)
print(X_test.shape)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from keras.models import Model
import keras
restnet = VGG16(include_top=False, weights='imagenet', input_shape=(255,255,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
vgg16 = Model(restnet.input, outputs=output)
for layer in restnet.layers:
  layer.trainable = False
vgg16.summary()

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from tensorflow.keras import optimizers
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])
model.summary()


In [ ]:
vgghist = model.fit(X_train2,y_train2, validation_data = (X_val,y_val), epochs=10)


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(255,255,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
resnet50 = Model(restnet.input, outputs=output)
for layer in restnet.layers:
  layer.trainable = False
resnet50.summary()

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from tensorflow.keras import optimizers
model = Sequential()
model.add(resnet50)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])
model.summary()


In [ ]:
resnet_his = model.fit(X_train2,y_train2, validation_data = (X_val,y_val), epochs=10)